## Preliminary K_Means

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

In [2]:
train = pd.read_csv('D:\\MasterBigData\\Didattica\\DMML\\Project\\training_ready.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IsBadBuy         72983 non-null  int64  
 1   Auction          72983 non-null  int64  
 2   VehicleAge       72983 non-null  int64  
 3   WheelType        72983 non-null  object 
 4   VehOdo           72983 non-null  int64  
 5   Nationality      72983 non-null  object 
 6   VehBCost         72983 non-null  float64
 7   WarrantyCost     72983 non-null  int64  
 8   OutCost          72983 non-null  int64  
 9   VehOdo_Standard  72983 non-null  float64
 10  AverageMMR       72983 non-null  float64
 11  Make_new         72983 non-null  object 
 12  VNST_new         72983 non-null  object 
 13  Size_class       72983 non-null  int64  
 14  Model_Name       72983 non-null  object 
 15  Model_Property   72983 non-null  object 
 16  SubModel_New     72983 non-null  object 
 17  Trim_New    

In [4]:
X = train[["VehicleAge", "VehOdo", "VehBCost", "WarrantyCost", "AverageMMR", "Size_class"]].values
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

Per ridurre la dimensionalità del dataset applichiamo un k-means con k = 20k

In [5]:
kmeans = KMeans(n_clusters = 2000, n_init=10, max_iter = 10, random_state = 1)
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10,
       n_clusters=2000, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=1, tol=0.0001, verbose=0)

In [6]:
labels, sizes = np.unique(kmeans.labels_, return_counts=True)
stats.describe(sizes)

DescribeResult(nobs=2000, minmax=(1, 272), mean=36.4915, variance=875.633744622311, skewness=2.052365107594514, kurtosis=8.140863163516181)

In [22]:
train["label"] = kmeans.labels_
q = train.groupby("label").mean()
q.drop(['OutCost', 'VehOdo_Standard'], axis=1, inplace=True)
q.head()

,IsBadBuy,Auction,VehicleAge,VehOdo,VehBCost,WarrantyCost,AverageMMR,Size_class
label,,,,,,,,
0,0.071429,0.728571,5.0,80436.185714,4370.571429,999.900000,4040.810714,1.0
1,0.300000,0.733333,6.0,77877.800000,6603.666667,1006.866667,6305.958333,6.0
2,0.041667,0.875000,2.0,45059.402778,7084.861111,881.069444,8373.621528,1.0
3,0.092437,0.789916,3.0,68876.378151,6979.592437,836.050420,9033.485294,1.0
4,0.230769,0.692308,4.0,77955.000000,7081.153846,688.538462,8326.615385,6.0


In [19]:
c = train.groupby(['label']).agg(lambda x:x.value_counts().index[0])

In [20]:
c.drop(['IsBadBuy','Auction', 'VehicleAge', 'VehOdo', 'VehBCost', 'WarrantyCost', 'OutCost', 'VehOdo_Standard', 'AverageMMR', 'Size_class'],
      inplace=True, axis=1)
c.head()

,WheelType,Nationality,Make_new,VNST_new,Model_Name,Model_Property,SubModel_New,Trim_New
label,,,,,,,,
0,Alloy,AMERICAN,FORD,AZ,TAURUS,3.0L,SEDAN,SE
1,Alloy,OTHER ASIAN,HYUNDAI,FL,SANTA,2WD,SPORT,SE
2,Covers,AMERICAN,CHRYSLER,AZ,SEBRING,4C,SEDAN,Bas
3,Covers,AMERICAN,DODGE,FL,CALIBER,None,SEDAN,L
4,Alloy,OTHER ASIAN,HYUNDAI,TX,SANTA,FE,SPORT,GL


In [37]:
q['Row'] = q.index
c['Row'] = c.index
df = q.join(c.set_index('Row'), on='Row')
df.drop(['Row'], axis=1, inplace=True)
df

,IsBadBuy,Auction,VehicleAge,VehOdo,VehBCost,WarrantyCost,AverageMMR,Size_class,WheelType,Nationality,Make_new,VNST_new,Model_Name,Model_Property,SubModel_New,Trim_New
label,,,,,,,,,,,,,,,,
0,0.071429,0.728571,5.0,80436.185714,4370.571429,999.900000,4040.810714,1.0,Alloy,AMERICAN,FORD,AZ,TAURUS,3.0L,SEDAN,SE
1,0.300000,0.733333,6.0,77877.800000,6603.666667,1006.866667,6305.958333,6.0,Alloy,OTHER ASIAN,HYUNDAI,FL,SANTA,2WD,SPORT,SE
2,0.041667,0.875000,2.0,45059.402778,7084.861111,881.069444,8373.621528,1.0,Covers,AMERICAN,CHRYSLER,AZ,SEBRING,4C,SEDAN,Bas
3,0.092437,0.789916,3.0,68876.378151,6979.592437,836.050420,9033.485294,1.0,Covers,AMERICAN,DODGE,FL,CALIBER,None,SEDAN,L
4,0.230769,0.692308,4.0,77955.000000,7081.153846,688.538462,8326.615385,6.0,Alloy,OTHER ASIAN,HYUNDAI,TX,SANTA,FE,SPORT,GL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.285714,0.678571,5.0,59890.500000,7122.142857,670.000000,7381.571429,5.0,Alloy,AMERICAN,JEEP,TX,LIBERTY,2WD,SUV,S
1996,0.125000,0.833333,5.0,62435.958333,8278.333333,1152.062500,9004.718750,6.0,Alloy,AMERICAN,CHEVROLET,TX,EQUINOX,2WD,SUV,L
1997,0.000000,1.000000,7.2,51376.600000,7596.000000,1013.800000,7289.750000,5.0,Covers,AMERICAN,JEEP,AZ,LIBERTY,4WD,SUV,S


In [38]:
df.to_csv('D:\\MasterBigData\\Didattica\\DMML\\Project\\training_clustered.csv',index=False)